# Prerequisites
본 `ipynb` 은 `Python=3.12` 에서 작성하였습니다. Package dependency 를 해결하기 위해 아래 cell 을 실행해주세요.

## Install Python packages

In [ ]:
%pip -q install -U dotenv openai azure-ai-projects azure-monitor-opentelemetry opentelemetry-instrumentation-openai-v2

## Load environment variables from a .env file
secret 노출을 피하고 notebook 들간의 일관된 환경변수를 설정하기 위해 `dotenv` 을 이용한다.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_AI_FOUNDRY_PROJECT_ENDPOINT = os.getenv("AZURE_AI_FOUNDRY_PROJECT_ENDPOINT")

# Connect azure monitor of AI Foundry project

In [ ]:
# input/output message content를 캡처하기 위해 환경변수를 설정합니다.
import os

os.environ["OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT"] = "true"
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"

# from azure.ai.projects import AIProjectClient
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor
from azure.ai.projects import AIProjectClient
from azure.identity import AzureCliCredential

# OpenAI API 호출을 자동으로 계측하기 위해 OpenAIInstrumentor를 사용합니다.
OpenAIInstrumentor().instrument()

# AIProjectClient를 사용하여 Azure AI Foundry 프로젝트에 연결합니다.
# API Key를 TokenCredential로 변환합니다.
project_client = AIProjectClient(
    credential=AzureCliCredential(),
    endpoint=AZURE_AI_FOUNDRY_PROJECT_ENDPOINT,
)
# Azure Monitor를 구성합니다.
configure_azure_monitor(connection_string=project_client.telemetry.get_application_insights_connection_string())

# OpenAI Generative APIs
OpenAI 는 Frontier LLM 모델을 서비스한다. Azure 는 OpenAI 의 동일한 모델들과 APIs 를 서비스하고 있으며, Python 에서는 `openai` package 를 통해서 Azure OpenAI (AOAI) 의 client 를 생성할 수 있다.

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)

## Simply Q&A Behavior
가장 초기에 나온 chat completions API 를 통해 question 에 대한 answer 를 synthesis 해보자.

In [ ]:
# [required] 꼭 gpt-4.1 모델이 Azure OpenAI 에서 배포되어 있어야 한다.
r = client.chat.completions.create(
    model="gpt-4.1",
    messages=[{"role": "user", "content": "GPT 를 쓸때 비용을 절감할 수 있는 방법은 어떤게 있을까 ?"}],
)

print("=== Response Message ===")
print(r.choices[0].message.content)
print("\n=== Response Usage ===")
print(r.usage.model_dump_json(indent=2))

## Advanced Q&A using Assistants API
Assistants API 는 Conversational API 이다. 단순 answer 를 생성해내는 것을 넘어 사용자와 모델의 대화를 제어할 수 있다. 여전히 beta 이고 벌써 deprecated 된단다.

In [ ]:
# assistants API 를 통해 Q&A 를 수행합니다.
assistant = client.beta.assistants.create(model="gpt-4.1")

thread = client.beta.threads.create()
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="GPT 를 쓸때 비용을 절감할 수 있는 방법은 어떤게 있을까 ?",
)

run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

# 폴링 (간단 예시)
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run.status == "completed":
        break

messages = client.beta.threads.messages.list(thread_id=thread.id)
print("=== Response Message ===")
print(messages.data[0].content[0].text.value)
print("\n=== Run Usage ===")
print(run.usage.model_dump_json(indent=2))

## Other approach Q&A using Responses API
가장 최신에 공개된 응답형 API 이다. 각 dialog 마다 `response id` 가 주어지고 이를 통해 conversation history 를 제어할 수 있다.

In [ ]:
# Responses API 를 통해 Q&A 를 수행합니다.
r = client.responses.create(
    model="gpt-4.1",
    input=[{"role": "user", "content": "GPT 를 쓸때 비용을 절감할 수 있는 방법은 어떤게 있을까 ?"}],
    previous_response_id=None,
)

print("=== Response Message ===")
print(r.output_text)
print("\n=== Response Usage ===")
print(r.usage.model_dump_json(indent=2))